# Imports et setup
## Imports des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import timeit

from pandas.plotting import register_matplotlib_converters
from pandas.api.types import union_categoricals, CategoricalDtype

## Setup des outils de visualisation

In [ ]:
%matplotlib inline
pd.set_option('display.max_columns', None)
register_matplotlib_converters()
sns.set(style="ticks", color_codes=True)

# Chargement du Dataset

## Définition des fichiers

In [ ]:
# historiques de vente
file_list = ['./Data/LDV_CONV_1ALO_201707_201906_V3.csv',
             './Data/LDV_CONV_1ALO_V3_20190909.csv', 
             './Data/LDV_CONV_1ALO_V3_20190916.csv', 
             './Data/LDV_CONV_1ALO_V3_20191011.csv',
             './Data/LDV_CONV_1ALO_V3_20191024.csv']

client_filename = './Data/Référentiel_ConverteO_1ALO_Clt_20191024.csv'
material_filename = './Data/Référentiel_ConverteO_1ALO_Art_20191024.csv'
scope_filename = './Data/split_cli_test_reco.csv'

# Pour le moment, on n'intègre que le dernier fichier de reco
previous_reco_filename = './Data/Recos_KNN_post_filtres_20191024.csv'

## Historiques de vente

Définition du format, et de l'index cible (une fois la concaténation effectuée).

In [ ]:
fields = {'orgacom':'category',
          'month':'category',
          'week':'category',
          'date':'object',
          'pricetype':'category',
          'client':'object',
          'doctype':'category',
          'origin':'category',
          'salesgroup':'category',
          'material':'object',
          'brutrevenue':'float',
          'brutrevcur':'category', 
          'netrevenue':'float', 
          'netrevcur':'category',
          'weight':'float',
          'weightunit':'category',
          'marginperkg':'float'}

Définition d'une fonction qui permet de concaténer les lignes d'historiques sur des fichiers transmis. Elle permet de concaténer des Dataframes (df1, df2, ..., dfN) dans l'ordre, avec la règle de gestion suivante : si une date est présente sur au moins une ligne du Dataframe df(n+1), toutes les lignes avec cette date sont supprimées du Dataframe df(n) avant concaténation.

C'est l'argument 'concat_index' qui va permettre d'identifier sur quelle donnée effectuer ces filtres successifs. Si 'concat_index' est passé avec 'date', alors si une date est présente dans un fichier alors qu'elle était dans un des fichiers précédents, alors elle est droppée (au profit du contenu du nouveau fichier).

De plus, les catégories sont alignées au fil de l'eau afin que la concaténation ne se traduisent pas par un upcast vers 'object'.

In [ ]:
def concat_df(file_list, **kwargs):
    for file_path in file_list:
        if 'df' not in locals():
            print('Loading ' + file_path)
            df = pd.read_csv(file_path, **kwargs)
        else:
            print('Loading ' + file_path)
            df2 = pd.read_csv(file_path, **kwargs)
            for field, my_type in fields.items():
                if my_type == 'category':
                    uc = union_categoricals([df[field],df2[field]])
                    df[field] = pd.Categorical( df[field], categories=uc.categories )
                    df2[field] = pd.Categorical( df2[field], categories=uc.categories )
            df = pd.concat([df[~df.index.isin(df2.index)] ,df2])
    print('Done!')
    return(df)

Chargement des fichiers du dataset :

In [ ]:
concat_index = ['date']

df = concat_df(file_list,
               sep=';', 
               header=None, 
               names=fields.keys(), 
               dtype=fields, 
               parse_dates=['date'], 
               index_col=concat_index)

In [ ]:
df.info()

In [ ]:
df.head()

On détruit l'index 'date', qui n'a que peu de sens.

In [ ]:
df.reset_index(inplace=True)
df.head()

## Clients

### Chargement

On commence par charger le dataset.

In [ ]:
fields = {'code client':'object',
          'libellé client':'object',
          'code catégorie client':'category',
          'libellé catéorie client':'category', 
          'KNA1-KATR5':'category',
          'KNA1-LOEVM':'category',
          'KNVV-LOEVM':'category',
          'KNVV-PLTYP':'category',
          'colonne_source_reco':'category',
          'GrVd':'category',
          'OrgCm':'category',
          'CDis':'category', 
          'Groupe':'category', 
          'P.':'category',
          'Cde postal':'category', 
          'KNA1-KATR1':'category', 
          'KNA1-KATR2':'category', 
          'KNA1-KATR3':'category', 
          'KNA1-KATR4':'category', 
          'KNA1-KATR6':'category'}

df_clt = pd.read_csv(client_filename, 
                     sep=';', 
                     header=0, 
                     encoding="ISO-8859-1", 
                     dtype=fields)
df_clt.head()

In [ ]:
df_clt.info()

On ajoute les zéros au niveau du code client, pour pouvoir joindre avec la table des historiques de vente.

In [ ]:
df_clt['code client'] = df_clt['code client'].apply(lambda x: x.zfill(10))

Ajout et tri de l'index

In [ ]:
df_clt.set_index('code client', inplace=True)
df_clt.sort_index(inplace=True)

On contrôle qu'il n'y a pas de code client en double.

In [ ]:
if np.any(df_clt.index.duplicated(keep=False)):
    raise RuntimeError('Attention ! Il existe des doublons sur l\'index du DataFrame df_clt')

In [ ]:
df_clt.info()

### Analyse

On peut vérifier par exemple la répartition des clients pour chacun des différents niveaux de segmentation :

In [ ]:
new_index = ['KNA1-KATR1', 'KNA1-KATR2', 'KNA1-KATR3', 'KNA1-KATR4', 'KNA1-KATR5', 'KNA1-KATR6', 'code client']
df_clt.reset_index().set_index(new_index).sort_index().head(15)

Si on regarde uniquement la répartition des clients par segment, jusqu'à la restauration commerciale indépendante, et enfin en y filtrant les traiteurs (catégorie ZY) on obtient : 

In [ ]:
segments = ['Z3', 'Z5', 'ZK', 'ZG']
filter = ['df_clt[\'KNA1-KATR' + str(i) + '\'] == \'' + segments[i-1] + '\', \'KNA1-KATR' + str(i+1) + '\'' for i in range(1, 5)]
filter

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(7,20))
fig.subplots_adjust(hspace=0.5)

temp_df = df_clt.loc[:, 'KNA1-KATR1'].value_counts().sort_index()
colors = ['C0'] * len(temp_df.index)
colors[temp_df.index.get_loc(segments[0])] = 'green'
temp_df.plot(kind='bar', ax=axs[0], color=colors)
axs[0].set_title('Racine', fontsize=18)

for i in range(1, 5):
    temp_df = df_clt.loc[eval(filter[i-1])].value_counts().sort_index().loc[lambda x: x>0]
    colors = ['C0'] * len(temp_df.index)
    if i<4:
        colors[temp_df.index.get_loc(segments[i])] = 'green'
    else: 
        colors = ['green'] * len(temp_df.index)
        colors[temp_df.index.get_loc('ZY')] = 'red'
    temp_df.plot(kind='bar', ax=axs[i], color=colors)
    axs[i].set_title('Enfant de ' + ' - '.join(segments[:i]), fontsize=18)

## Articles

In [ ]:
fields = {'code article':'object',
          'libellé article':'category',
          'code gamme':'category',
          'libellé gamme':'category', 
          'MARC-MMSTA':'category',
          'MARC-LVORM':'category',
          'MVKE-LVORM':'category',
          'MVKE-MVSTA':'category',
          'MARA-LVORM':'category',
          'Hiérarchie produit':'category',
          'Type d\'article':'category',
          'Division':'category', 
          'Org. commerciale':'category', 
          'Canal distribution':'category',
          'File d\'achat':'category', 
          'Marque industrielle':'category', 
          'Marque commerciale':'category', 
          'Grpe de marchandises':'category', 
          'Poids net':'float', 
          'Unité de p':'category',
          'V1':'category',
          'V2':'category',
          'V3':'category',
          'LG1':'category',
          'LG2':'category',         
         }

df_mat = pd.read_csv(material_filename, 
                     sep=';', 
                     header=0, 
                     encoding="ISO-8859-1", 
                     dtype=fields,
                     decimal=",")
df_mat.head()

In [ ]:
df_mat['code article'] = df_mat['code article'].apply(lambda x: x.zfill(18))

In [ ]:
df_mat.set_index('code article', inplace=True)

In [ ]:
df_mat.head()

## Périmètre client

On récupère le périmètre client, et on ajoute cette info au dataset client.

In [ ]:
fields = {'code article':'object',
          'GrVd':'category',
          'code_client':'object', 
          'marge_livraison_moy_cli':'float',
          'rank_MLV_intra_GV':'int64', 
          'flag_reco':'int64',
          'groupe_test':'category'
         }

df_perim_clt = pd.read_csv(scope_filename, 
                           sep=';', 
                           header=0, 
                           dtype=fields)
df_perim_clt.info()

In [ ]:
df_clt.head()

In [ ]:
df_perim_clt['code_client'] = df_perim_clt['code_client'].str.zfill(10)
df_perim_clt.set_index('code_client', inplace=True)
df_perim_clt.head()

In [ ]:
df_perim_clt.info()

In [ ]:
df_clt['flag_reco'] = 0
df_clt.update(df_perim_clt['flag_reco'])
df_clt['flag_reco'] = df_clt['flag_reco'].astype(np.int64)
df_clt.info()

In [ ]:
df_clt.groupby('flag_reco').size()

In [ ]:
del df_perim_clt

## Recommandations précédentes

On commence par charger le fichier.

In [ ]:
fields = {'Code_client':'object',
          'Code_article':'object',
          'xcom':'category',
          'xdelais':'category',
          'mailDemandeur':'category',
          'libelleClient':'object',
          'libelleArticle':'object',
          'origine':'category',
          'rang':'int64',
          'rating':'float64'
         }

df_prev_reco = pd.read_csv(previous_reco_filename, 
                          sep=';', 
                          header=0, 
                          dtype=fields,
                          encoding="ISO-8859-1")

df_prev_reco.info()

On met à jour le code article (zéro fillé sur 18 digits).

On récupère certains champs du dataframe article (pour le moment, la hiérarchie produit).

In [ ]:
df_prev_reco['Code_article'] = df_prev_reco['Code_article'].apply(lambda x: x.zfill(18))
df_prev_reco = df_prev_reco.set_index('Code_article').join(df_mat['Hiérarchie produit']).reset_index()
df_prev_reco.rename(columns={'index':'Code_article'}, inplace=True)
df_prev_reco.head()

## Merges

In [ ]:
df = df.merge(df_clt, how='left', left_on=['client'], right_index=True, validate='m:1')
df = df.merge(df_mat, how='left', left_on='material', right_index=True)
df.info()

In [ ]:
df.head()

## Filtre sur les types de documents à conserver

On ne conserve que les lignes qui concernent des commandes de vente.

In [ ]:
filtered_doctypes = ['ZC01', 'ZC02', 'ZC10']
df = df[df.doctype.isin(filtered_doctypes)].copy()
cat_type = CategoricalDtype(categories=['ZC10', 'ZC01', 'ZC02'], ordered=True)
df['doctype'] = df['doctype'].astype(cat_type)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.groupby(by='doctype').size()

In [ ]:
del df_clt
del df_mat

## Filtre sur les lignes de traiteurs

On retire les lignes de traiteur

In [ ]:
color = ['green'] * 9
color[8] = 'red'
df.loc[:, 'KNA1-KATR5'].value_counts().sort_index().loc[lambda x: x>0].plot(kind='bar', color=color)

Remarque : cette fois, on n'est plus sur un nombre de clients, mais sur un nombre de lignes dans le dataset d'historiques de vente.

In [ ]:
df.drop(df[df['KNA1-KATR5'] == 'ZY'].index, inplace=True)
df.loc[:, 'KNA1-KATR5'].value_counts().sort_index().loc[lambda x: x>0].plot(kind='bar', color=color)

Les lignes du dataset concernant les traiteurs on disparu.

## Analyse des premiers ratios

In [ ]:
df.describe()

On voit que certains ratios sont nuls (CA et poids), ce qui risque de poser problème lors du calcul de nouveaux indicateurs (ex : prix de vente au kg, marge %, ...)

On va commencer par analyser les incohérences potentielles, en identifiant les relations pour lesquelles les ratios nuls sont incohérents entre eux.

On abandonne la notion de CA net net, qui est une donnée purement "gestion" et pas commerciale.

In [ ]:
cat_type = pd.api.types.CategoricalDtype(categories=['neg', 'nul', 'pos'],
                            ordered=True)

df['rev_cat'] = 'pos'
df.loc[df['brutrevenue'] == 0, 'rev_cat'] = 'nul'
df['rev_cat'] = df['rev_cat'].astype(cat_type)
df['wei_cat'] = 'pos'
df.loc[df['weight'] == 0, 'wei_cat'] = 'nul'
df['wei_cat'] = df['wei_cat'].astype(cat_type)
df['mrg_cat'] = 'pos'
df.loc[df['marginperkg'] == 0, 'mrg_cat'] = 'nul'
df.loc[df['marginperkg'] < 0, 'mrg_cat'] = 'neg'
df['mrg_cat'] = df['mrg_cat'].astype(cat_type)
df.head()

In [ ]:
df.groupby(by=['rev_cat', 'wei_cat', 'mrg_cat'])['material'].count().unstack()

Si on regarde les ratios ci-dessus, partant des plus représentés : 

In [ ]:
df.groupby(by=['rev_cat', 'wei_cat', 'mrg_cat'])['material'].count().sort_values(ascending=False)

pos / pos / pos : on a un CA, un poids et une marge positive. Parfait.

nul / nul / nul : on dirait des annulations de ligne. On va supprimer ces lignes du dataset. Cf. le résultat de la requête : 
df.loc[(df['client'] == '0000262869')].set_index(['material', 'date']).sort_index() : on voit que les lignes "nulles" sont le pendant de lignes commandées.

pos / pos / neg : c'est quand on vend mal... on les garde.

nul / pos / neg : on dirait des gratuits. On garde.

pos / pos / pos : on vend plutôt mal, avec une marge nulle. On garde.

nul / pos / pos : cas bizarre, on dirait des postes de gratuit, avec une erreur sur le PRN ? Ne concerne que l'article 197832. On droppe.

pos / nul / nul : article de service, forfait livraison. On droppe également.

nul / pos / nul : commande échantillon, sur un unique produit. Le PRN n'était peut être pas à jour. On droppe.

In [ ]:
#Cette cellule permet de contrôler le contenu du dataset pour les différentes combinaisons.
df.loc[(df['rev_cat'] == 'nul') & (df['wei_cat'] == 'pos') & (df['mrg_cat'] == 'nul')]

In [ ]:
df = df.loc[~((df['rev_cat'] == 'nul') & (df['wei_cat'] == 'nul') & (df['mrg_cat'] == 'nul'))]
df = df.loc[~((df['rev_cat'] == 'pos') & (df['wei_cat'] == 'nul') & (df['mrg_cat'] == 'nul'))]
df = df.loc[~((df['rev_cat'] == 'nul') & (df['wei_cat'] == 'pos') & (df['mrg_cat'] == 'pos'))]
df = df.loc[~((df['rev_cat'] == 'nul') & (df['wei_cat'] == 'pos') & (df['mrg_cat'] == 'nul'))].copy()

In [ ]:
df.groupby(by=['rev_cat', 'wei_cat', 'mrg_cat'])['material'].count().sort_values(ascending=False)

# Calcul de la marge (sur CA brut)

In [ ]:
df['margin'] = df['weight'] * df['marginperkg']
df.head()

In [ ]:
df.describe()

In [ ]:
sns.pairplot(df.loc[:, ['brutrevenue', 'weight', 'margin']])

On voit que la représentation du dataset est complètement écrasée par les outliers.

On va clipper ces outliers, sur ces 3 ratios, puis recalculer les ratios initiaux (en particulier, marge en €/kg). 

On calcule d'abord les valeurs limites pour les ratios CA brut, Poids du poste et Marge du poste, en bornant à 3 écarts-types.

In [ ]:
def clipvalue(series, stdcount=3):
    return(series.mean() + stdcount * series.std())

ratios = ['brutrevenue', 'weight', 'margin']
clipvals = {ratio : clipvalue(df.loc[:, ratio], stdcount=3) for ratio in ratios}

print(clipvals)

On calcule les nouveaux ratios, et on compare aux ratios initiaux.

In [ ]:
for ratio, clipval in clipvals.items():
    print(ratio + ' en cours de traitement')
    print(clipval)
    df[ratio + '_clipped'] = df[ratio].clip(lower=-clipval, upper=clipval)

# on recalcule la marge au kg pour avoir de la cohérence
df['marginperkg_clipped'] = df['margin_clipped'] / df['weight_clipped']
    
df=df.copy()
df[['brutrevenue', 'brutrevenue_clipped', 'weight', 'weight_clipped', 'margin', 'margin_clipped', 'marginperkg', 'marginperkg_clipped']].describe()

Les valeurs moyennes des ratios sont légèrement plus faibles, les écarts types et les valeurs max se sont fortement réduits.

On va à nouveau tenter de représenter la distribution des points.

In [ ]:
sns.pairplot(df.loc[:, ['brutrevenue_clipped', 'weight_clipped', 'margin_clipped']])

La représentation comporte trop de points pour pouvoir conclure. On analysera un peu plus dans un autre chapitre.

# Calcul du PMVK

On calcule le prix moyen de vente au kilo sur la base des valeurs clippées.

In [ ]:
df['pmvk'] = df['brutrevenue_clipped'] / df['weight_clipped']
df['pmvk'].describe()

In [ ]:
sns.kdeplot(df['pmvk'])

On a à nouveau des outliers sur cette nouvelle donnée, qui "écrasent" la distribution. Si on regarde les pmvk les plus gros : 

In [ ]:
df.sort_values('pmvk', ascending=False).head(20)

Au-delà de quelques erreurs de prix, on voit que l'article 9800085 Conservateur GT1702 vitre est sur représenté. Il s'agit d'un article de PLV (une sorte de frigo), qui coûte 495€ pour lequel le poids de la ligne remonte à 1kg (d'où un pmvk décalé). Il s'agissait d'une erreur sur le poids de l'article, qui a été ensuite corrigée.

On va à nouveau clipper cette donnée, sachant qu'il y aura une incohérence sur les lignes corrigées entre PMVK, CA brut et poids (on ne souhaite pas modifier CA brut ou poids).

In [ ]:
pmvk_clip = clipvalue(df.loc[:, 'pmvk'], stdcount=3)
pmvk_clip

In [ ]:
df['pmvk_clipped'] = df['pmvk'].clip(lower=-pmvk_clip, upper=pmvk_clip)
df[['pmvk', 'pmvk_clipped']].describe()

In [ ]:
sns.kdeplot(df['pmvk_clipped'])

Au-delà de l'artefact de droite dû au "clippage", on a une courbe plutôt irrégulière...

# Analyse détaillée des ratios entre eux

On passe sur une visualisation de la densité du noyau pour mieux voir la répartition des points sur le dataset.

In [ ]:
g = sns.PairGrid(df.loc[:, ['brutrevenue_clipped', 'weight_clipped', 'margin_clipped', 'pmvk_clipped']].sample(5000))
g = g.map_offdiag(sns.kdeplot, shade=True, shade_lowest=False)
g = g.map_diag(sns.distplot)

On voit que même après avoir réduit les extremums, les points sont concentrés dans un tout petit espace. De plus, la répartition des poids des postes montre des irrégularités curieuses.

Si on zoome sur les zones représentatives, on obtient la visualisation suivante :

In [ ]:
g.axes[0,1].set_ylim(-5, 65)
g.axes[0,2].set_xlim(-3, 20)
g.axes[1,0].set_ylim(-1, 13)
g.axes[1,0].set_xlim(-5, 50)
g.axes[2,1].set_ylim(-3, 20)
g.axes[2,1].set_xlim(-1, 7)
g.axes[0,3].set_xlim(-3, 17)
g.fig

Globalement, on voit bien que les données sur les 3 premiers ratios semblent linéairement corrélées (les tâches sont plutôt le long d'une droite y = ax). Néanmoins, il y a une "verrue", qu'on voit par exemple aux alentours de weight = 5 et brutrevenue = 15.

Les irrégularités sur la courbe du poids (le graphe central, sur lequel on voit un second maximum local) sont sur des lignes qui doivent avoir un PMVK qui est plus faible (poids plus important, mais CA à peu près similaire).

In [ ]:
hue_var = 'Grpe de marchandises'
sample_size = 5000

df_plot = df.loc[:, ['brutrevenue_clipped', 'weight_clipped', 'margin_clipped', 'pmvk_clipped', hue_var]].sample(sample_size)
print(df_plot.groupby(hue_var).count())
print(df_plot[hue_var].cat.categories)
df_plot[hue_var].cat.remove_unused_categories(inplace=True)
print(df_plot[hue_var].cat.categories)
df_plot.head()

In [ ]:
#g2 = sns.PairGrid(df_plot, hue=hue_var)
#g2 = g2.map_offdiag(sns.kdeplot,shade=True, shade_lowest=False, alpha=0.5)
#g2 = g2.map_diag(sns.kdeplot)


In [ ]:
#g2.axes[0,1].set_ylim(-5, 65)
#g2.axes[0,2].set_xlim(-3, 20)
#g2.axes[1,0].set_ylim(-1, 13)
#g2.axes[1,0].set_xlim(-5, 50)
#g2.axes[2,1].set_ylim(-3, 20)
#g2.axes[2,1].set_xlim(-1, 7)
#g2.axes[0,3].set_xlim(-3, 17)
#g2 = g2.add_legend()
#g2.fig

# Analyse par jour de la semaine

In [ ]:
df['weekday'] = df['date'].dt.weekday + 1
df.tail()

In [ ]:
#print(df.groupby('weekday')['client'].count())
ax = df.groupby('weekday')['client'].count().plot(kind='bar')

Pour éviter d'avoir des effets de bord lors de l'affichage temporel, on droppe les lignes qui concernent des samedis.

In [ ]:
df.groupby('weekday').size()

In [ ]:
df = df[df['weekday'] != 6].copy()

In [ ]:
df.groupby('weekday').size()

On calcule le nombre de postes par jour et le nombre de commandes par jour. On peut d'abord définir une liste d'axes complémentaires pour l'analyse, en listant les critères dans la liste suivantes.

In [ ]:
crit_list = ['flag_reco']

In [ ]:
df_postes = df.groupby(['date'] + crit_list).size()
df_postes.head()

In [ ]:
df_commandes = df.groupby(['date', 'client'] + crit_list).size().groupby(['date'] + crit_list).size()
df_commandes.head()

In [ ]:
df_commandes = pd.concat([df_commandes, df_postes], axis=1)
df_commandes.head()

In [ ]:
df_commandes.columns = ['commandes', 'postes']

In [ ]:
df_commandes['nb_moy_lig'] = df_commandes['postes'] / df_commandes['commandes']
df_commandes.reset_index(inplace=True)
df_commandes.head()

In [ ]:
df_agg = df_commandes[['commandes', 'postes', 'date']].groupby('date').sum()
df_agg['nb_moy_lig'] = df_agg['postes'] / df_agg['commandes']
ax = df_agg['nb_moy_lig'].plot(kind='line', figsize=(13, 8))


In [ ]:
ax = df_agg.rolling(window=15, min_periods=0).mean()['nb_moy_lig'].plot(kind='line', figsize = (13, 8))

Il semblerait qu'il y ait une saisonnalité au niveau de cet indicateur. On affiche en empilant les données année par année pour se faire une idée.

In [ ]:
df_agg.reset_index(inplace=True)
df_agg['year'] = df_agg['date'].apply(lambda x: x.year)
df_agg['dayofyear'] = df_agg['date'].apply(lambda x: x.dayofyear)
df_agg.head()

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
for year_ in [2017, 2018, 2019]:
    df_plot = df_agg.loc[df_agg['year'] == year_].reset_index(drop=True).rolling(window=15, min_periods=3).mean()
    line, = ax.plot(df_plot['dayofyear'], df_plot['nb_moy_lig'])
    line.set_label(str(year_))
ax.legend()
ax.set_title("Nombre de lignes par commande - Comparatif entre années", fontsize=20)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))
labels = ['Sans recos', 'Avec recos']
for i in range(2):
    df2 = df_commandes.loc[df_commandes.flag_reco == i].set_index('date').rolling(window=15, min_periods=3).mean().reset_index()
    line, = ax.plot(df2['date'].dt.to_pydatetime(), df2['nb_moy_lig'])
    line.set_label(labels[i])
ax.legend()
ax.axvline(x=pd.to_datetime('20190930'), color='green')
#ax.set_xlim(pd.to_datetime('20181101'))
ax.set_title("Nombre de lignes par commande - Comparatif A/B test", fontsize=20)

In [ ]:
ax.set_ylim(0)
ax.figure

# Analyse des recos "trop similaires"

On va identifier les recommandations "trop similaires" à des produits déjà récurrents.

On commence par calculer les produits récurrents par client.

In [ ]:
df['rank_commandes_par_resto'] = df.groupby('client')['date'].rank('dense', ascending=False)
ds3 = df.loc[df['rank_commandes_par_resto']<=12].groupby(['client', 'material']).size()
ds3 = ds3.rename('order_count_last_12')
ds3

In [ ]:
df = df.merge(ds3.reset_index(), how='left', on=['client', 'material'], validate='m:1')
df['order_count_last_12'].fillna(0, inplace=True)
df.head()

In [ ]:
# A Voir, à priori inutile
#df2 = df[df.groupby(['client', 'Hiérarchie produit'], observed=True)['order_count_last_12'].transform('max').eq(df['order_count_last_12'])]
#df2[df2['order_count_last_12']>=3].tail()

#df2 = df[df['order_count_last_12'] >= 3]
#df3 = df_prev_reco.merge(df2, 
#                         how='inner',
#                         left_on=['Code_client', 'Hiérarchie produit'],
#                         right_on=['client', 'Hiérarchie produit'])

#df3.head()

On construit une série qui permet de garder le nombre d'articles présents dans l'historique de chaque noeud de hiérarchie.

In [ ]:
myIndex = pd.Index([''])
uniq = df['material'].nunique()
ds = pd.Series(data=[uniq], index=myIndex)
for myLen in range(1,7):
    df['H'+str(myLen)] = df['Hiérarchie produit'].apply(lambda x: x[:(myLen*2)])
    ds = pd.concat([ds, df.groupby(['material', 'H' + str(myLen)]).size().groupby('H' + str(myLen)).size()])
ds.sort_index(inplace=True)
ds.head(25)

On écrit une fonction qui retourne la distance entre 2 articles.

In [ ]:
def mat_dist(mat1, mat2):
    if mat1 == mat2:
        return(0)
    else:
        myLen = 0
        h1, h2 = df[df['material'] == mat1].iloc[0]['H6'], df[df['material'] == mat2].iloc[0]['H6']
        while(h1[:(myLen*2)] == h2[:(myLen*2)]) and myLen < 6:
            myLen += 1
        return(ds[h1[:((myLen-1)*2)]])    


In [ ]:
mat_dist('000000000000000433','000000000000000433')

In [ ]:
import datetime
print(datetime.datetime.now())